## How to permanently add or burn subtitles to a video using Moviepy

For files (mp4 and srt) used in this repository check: https://github.com/ramsrigouthamg/Supertranslate.ai

Author: [Ramsri Goutham](https://twitter.com/ramsri_goutham)

srt to subtitle

In [ ]:
!pip install moviepy==2.0.0.dev2
!pip install imageio==2.25.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for moviepy: filename=moviepy-2.0.0.dev2-py3-none-any.whl size=111676 sha256=8289a93eb266830816fdafe55cb6c44118121fd6f8b484eaf8c938920aaefda0
  Stored in directory: /root/.cache/pip/wheels/90/13/30/9ccaa367cb61879edc46aa6ccb17484809aff8a3145f1adc07
Successfully built moviepy
  Attempting uninstall: moviepy
    Found existing installation: moviepy 1.0.3
    Uninstalling moviepy-1.0.3:
      Successfully uninstalled moviepy-1.0.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 33.2 MB/s eta 0:00:00
  Attempting uninstall: imageio
    Found existing installation: imageio 2.31.6
    Uninstalling imageio-2.31.6:
      Successfully uninstalled imageio-2.31.6


In [ ]:
!apt install imagemagick

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts imagemagick-6-common
  imagemagick-6.q16 libdjvulibre-text libdjvulibre21 libfftw3-double3 libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10 libwmflite-0.2-7 netpbm
  poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x imagemagick-doc
  autotrace cups-bsd | lpr | lprng enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer
  povray radiance sane-utils texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev
  inkscape poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic
  | fonts-ipafont-gothic fonts-arphic-ukai fonts-arphic-uming fonts-nan

In [ ]:
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

In [ ]:
!pip install pysrt==1.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13443 sha256=f9a4365efca5120f32ba29ea3c037e49744fae2254956238cc215c544af5b54f
  Stored in directory: /root/.cache/pip/wheels/30/7f/e8/55de9a9b07302d9e7fe47c27910e3bea0c48536153e74bd7e6
Successfully built pysrt


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import unicodedata

def eliminar_tildes(s):
    return ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

In [ ]:
import sys
import pysrt
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip, clips_array
import numpy as np
import moviepy.video.fx as vfx
from moviepy.video.fx.all import scroll


def time_to_seconds(time_obj):
    return time_obj.hours * 3600 + time_obj.minutes * 60 + time_obj.seconds + time_obj.milliseconds / 1000


def create_subtitle_clips(subtitles, videosize,fontsize=160, font='Humor-Sans', color='yellow', debug = False):
    subtitle_clips = []

    for subtitle in subtitles:
        start_time = time_to_seconds(subtitle.start)
        end_time = time_to_seconds(subtitle.end)
        duration = end_time - start_time

        video_width, video_height = videosize

        text_clip = TextClip(eliminar_tildes(subtitle.text), fontsize=fontsize, font=font, color=color, stroke_color='black', stroke_width = 6,size=(video_width*3/4, None), method='caption').set_start(start_time).set_duration(duration)
        subtitle_x_position = 'center'
        subtitle_y_position = video_height* 1 / 2

        text_position = (subtitle_x_position, subtitle_y_position)
        subtitle_clips.append(text_clip.set_position(text_position))

    return subtitle_clips

def add_color_effect(clip):

    # Agregar efecto de movimiento
    clip_fx = clip.fx(vfx.colorx, 5)

    # Devolver el video con el efecto
    return clip_fx

def add_random_motion_effect(clip):

    # Definir una función para el movimiento aleatorio
    def random_motion(get_frame, t):
        frame = get_frame(t)
        h, w, _ = frame.shape
        return np.roll(np.roll(frame, np.random.randint(-2, 2), axis=0), np.random.randint(-2, 2), axis=1)

    # Crear un nuevo clip con el efecto de movimiento
    clip_fx = clip.fl(random_motion)

    # Devolver el video con el efecto
    return clip_fx

def apply_motion(clip):
    # Definir el movimiento horizontal y vertical
    moving_clip = clip.fx(scroll, h=lambda t: 50*2, w=lambda t: 30*4)  # Ajusta los valores según tus necesidades

    # Asegurarse de que no hay bordes negros recortando el clip
    cropped_clip = moving_clip.crop(x_center='center', y_center='center')

    return cropped_clip



In [ ]:
import os

avatares_genero = {
    "avatar0.jpg":"fem",
    "avatar1.jpg":"fem",
    "avatar2.jpg":"masc",
    "avatar3.jpg":"masc",
    "avatar4.jpg":"masc",
    "avatar5.jpg":"masc",
    "avatar6.jpg":"masc",
    "avatar7.jpg":"fem",
    "avatar8.jpg":"fem"
}

In [ ]:
import pandas as pd
path = '/content/drive/MyDrive/TuChisteDiario'
df_chistes = pd.read_csv(path + '/chistes_con_metadatos.csv')

In [ ]:
df_chistes[(~df_chistes['subs_creado'])&(df_chistes['video_creado'])]

In [ ]:
from moviepy.editor import *

# Chistes a crear
df_chistes_crear = df_chistes[(~df_chistes['subs_creado'])&(df_chistes['video_creado'])]

for index, row in df_chistes_crear.iterrows():
  chiste = f'chiste_{index}'
  if not os.path.exists(path + '/videos_subtitulos/' + chiste + '.mp4'):
    #numero del chiste
    numero_chiste = int(chiste.replace('chiste_',''))
    #numero avatar
    numero_avatar = numero_chiste % len(avatares_genero.keys())
    #avatar
    avatar = 'avatar' + str(numero_avatar) + '.jpg'
    #intro and outro
    intro_filename = path + '/intro_outro/intro.mp4'
    intro = VideoFileClip(intro_filename)
    intro = intro.resize(height=2048, width=1152)
    outro_filename = path + '/intro_outro/outro.mp4'
    outro = VideoFileClip(outro_filename)
    outro = outro.resize(height=2048, width=1152)

    for video in os.listdir(path + '/videos/' + chiste):
      if video.endswith(".mp4"):
        print(f'------- Creando video de {chiste} ---------')
        mp4filename = path + '/videos/' + chiste + '/' + video
        srtfilename_masc = path + '/audios_chistes/' + chiste + '/subs_masc.srt'
        srtfilename_fem = path + '/audios_chistes/' + chiste + '/subs_fem.srt'
        srtfilename = srtfilename_masc if avatares_genero[avatar] == 'masc' else srtfilename_fem
        # Load video and SRT file
        video = VideoFileClip(mp4filename)
        subtitles = pysrt.open(srtfilename)

        # add zooms
        #video = apply_motion(video)

        # Create subtitle clips
        subtitle_clips = create_subtitle_clips(subtitles,video.size)
        # Add subtitles to the video
        final_video = CompositeVideoClip([video] + subtitle_clips)

        concatenated_video = concatenate_videoclips([intro, final_video, outro], method="compose")

        concatenated_video.write_videofile(path + '/videos_subtitulos/' + chiste + '.mp4')


        df_chistes.loc[index,'subs_creado'] = True
        df_chistes.to_csv(path+'/chistes_con_metadatos.csv',index=False)

In [8]:
from IPython.display import HTML
from base64 import b64encode
output_video_file = '/content/drive/MyDrive/TuChisteDiario/videos_subtitulos/chiste_79.mp4'
mp4 = open(output_video_file,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=250 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.